Setting Environment and elements


In [1]:
import sys
import pdb
import time

import ee
import numpy as np
from numpy import ndarray
import pandas as pd


In [2]:
#Sign in to Earth Engin API    
ee.Initialize()

dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate('2017-01-01', '2021-12-31') 

In [3]:

BANDS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7','ST_B10']
SCALE = 30
NUM_FILES=4


Defining helper functions

In [4]:
#Read grid file
def parse_grid_file(grid_file: str) -> ndarray:
    df = pd.read_csv(grid_file)
    vals = df.values
    str_to_arr = lambda s: np.array([float(x) for x in s.split(',')])
    vals = np.array([[str_to_arr(x2) for x2 in x1] for x1 in vals])
    return vals

#Base on Landsat 8 pixel quality band(band QA_PIXEL) to select RS images
def cloudMask(image):
    qa = image.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    return image.updateMask(cloud.Not())

#Read landsat 8 band data
def get_band_data(img_data, band):
    return np.array(img_data.get(band).getInfo())

Read datas

In [5]:
grid = parse_grid_file("grid.csv")
num_rects = len(grid)
file_splits = np.linspace(0, num_rects, NUM_FILES + 1).astype('int')
print('File Splits: ', file_splits)

File Splits:  [   0 1922 3844 5766 7688]


Download Landsat 8 image data

In [6]:
dfs = []
for rect_id in range(num_rects):
    start_time = time.time()
    
    #Get the geometry of the current rectangle by grid file
    rect = ee.Geometry.Polygon(grid[rect_id].tolist(), None, False)
    rect_imgs = dataset.filterBounds(rect).map(cloudMask).select(BANDS)
    
    #Get the date of the images in the current rectangle
    dates = rect_imgs.map(
        lambda x: ee.Feature(None, {'date': x.date().format('YYYY-MM-dd')})
    ).aggregate_array('date').getInfo()
    #Define a function to reduce the region to a single value and get the mean pixel values
    def reduce_region(img):
        meanpixelvalue  = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=rect,
            scale=SCALE
        )
        return ee.Feature(None, {'meanpixelvalue': meanpixelvalue})
    #Get whole mean pixel values
    meanpixelvalues  = rect_imgs.map(
        lambda x: reduce_region(x)
    ).aggregate_array('meanpixelvalue').getInfo()
    
    #Create a dataframe with the mean pixel values and the rect id and date
    #
    #Note: The rect id and date are stored in the 'rect' and 'date' columns respectively.
    df = pd.DataFrame(data=meanpixelvalues)
    df['rect'] = rect_id
    df['date'] = dates
    dfs.append(df)

    print(f'Finished RectID={rect_id} in {time.time() - start_time:3f} seconds.')


Finished RectID=0 in 3.134853 seconds.
Finished RectID=1 in 1.936688 seconds.
Finished RectID=2 in 3.252609 seconds.
Finished RectID=3 in 2.583397 seconds.
Finished RectID=4 in 2.821002 seconds.
Finished RectID=5 in 1.736639 seconds.
Finished RectID=6 in 2.371762 seconds.
Finished RectID=7 in 2.134694 seconds.
Finished RectID=8 in 2.789610 seconds.
Finished RectID=9 in 2.285042 seconds.
Finished RectID=10 in 2.205167 seconds.
Finished RectID=11 in 1.728396 seconds.
Finished RectID=12 in 2.450281 seconds.
Finished RectID=13 in 1.838374 seconds.
Finished RectID=14 in 1.781291 seconds.
Finished RectID=15 in 1.779364 seconds.
Finished RectID=16 in 1.812142 seconds.
Finished RectID=17 in 1.771090 seconds.
Finished RectID=18 in 1.730155 seconds.
Finished RectID=19 in 1.765097 seconds.
Finished RectID=20 in 1.767011 seconds.
Finished RectID=21 in 2.829991 seconds.
Finished RectID=22 in 1.816769 seconds.
Finished RectID=23 in 2.237079 seconds.
Finished RectID=24 in 1.747214 seconds.
Finished R

Output the image data


In [7]:
output = pd.concat(dfs,ignore_index=True)
output.to_csv('landsat8RSdata.csv',index=False)